In [1]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CS610_AML/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1wyO35GiaMELX5uUD8ZenhccOlW8RA1Dl/CS610_AML


In [ ]:
import os, shutil, random
from pathlib import Path
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

DATASET_ROOT = '/content/drive/MyDrive/CS610_AML/yolo_traintestsplit'

In [ ]:
# for reproducibility
SEED = 42
random.seed(SEED)

CLASS_NAMES    = {0: "selfie",
                  1: "bystander"}

yaml generator

In [ ]:
yaml_text = f"""# selfie-taker detection dataset
path: {Path(DATASET_ROOT).resolve()}

train: images/train
val:   images/val
test:  images/test

nc: {len(CLASS_NAMES)}
names: {list(CLASS_NAMES.values())}
"""
(Path(DATASET_ROOT)/"data_selfie.yaml").write_text(yaml_text)
print("data_selfie.yaml created → ", Path(DATASET_ROOT)/"data_selfie.yaml")

data_selfie.yaml created → /content/drive/MyDrive/CS610_AML/yolo_traintestsplit/data_selfie.yaml


In [ ]:
# view yaml file

yaml_path = Path(DATASET_ROOT) / "data_selfie.yaml"
print(yaml_path.read_text())


# selfie-taker detection dataset
path: /content/drive/.shortcut-targets-by-id/1wyO35GiaMELX5uUD8ZenhccOlW8RA1Dl/CS610_AML/yolo_traintestsplit

train: images/train
val:   images/val
test:  images/test

nc: 2
names: ['selfie', 'bystander']



# Fine-tuning YOLOv11s

In [ ]:
!yolo task=detect mode=train \
     model=yolo11s.pt \
     data=/content/drive/MyDrive/CS610_AML/yolo_traintestsplit/data_selfie.yaml \
     epochs=100 \
     imgsz=640 \
     batch=16 \
     device=0 \
     freeze = 10 lr0=0.0001 weight_decay=0 cos_lr = True

Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/CS610_AML/yolo_traintestsplit/data_selfie.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train17, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspectiv

# Test Set Evaluation

In [ ]:
# get class counts for weighted precision/ recall computation

import yaml

with open("/content/drive/MyDrive/CS610_AML/yolo_traintestsplit/data_selfie.yaml", "r") as f:
    data_cfg = yaml.safe_load(f)
names = data_cfg['names']  # Dict like {0: 'selfie', 1: 'bystander'}
nc = len(names)


label_dir = "/content/drive/MyDrive/CS610_AML/yolo_traintestsplit/labels/test"
class_counts = [0] * nc

for fname in os.listdir(label_dir):
    if not fname.endswith(".txt"):
        continue
    with open(os.path.join(label_dir, fname)) as f:
        for line in f:
            class_idx = int(line.strip().split()[0])
            class_counts[class_idx] += 1

print("GT instances per class:", dict(zip(names, class_counts)))

GT instances per class: {'selfie': 43, 'bystander': 161}


In [ ]:
from ultralytics import YOLO

model = YOLO("runs/detect/train17/weights/best.pt")
metrics = model.val(data="/content/drive/MyDrive/CS610_AML/yolo_traintestsplit/data_selfie.yaml", split='test',agnostic_nms=True)

# Print mean precision, recall, mAP@0.5, mAP@0.5:0.95
print("Mean Precision [across class](mp):", metrics.box.mp)
print("Mean Recall [across class] (mr):", metrics.box.mr)
print("mAP@0.5:", metrics.box.map50)
print("mAP@0.5:0.95:", metrics.box.map)
print()
print("Mean Precision/ Recall per Class")
for i, name in metrics.names.items():
    p = metrics.box.p[i]
    r = metrics.box.r[i]
    ap50 = metrics.box.ap50[i]
    ap = metrics.box.ap[i]
    print(f"{name}: Precision={p:.3f}, Recall={r:.3f}, AP@0.5={ap50:.3f}, AP@0.5:0.95={ap:.3f}")


print()
print('WEIGHTED PRECISION/RECALL')
precisions = metrics.box.p
recalls = metrics.box.r
total_instances = sum(class_counts)

weighted_precision = sum(p * n for p, n in zip(precisions, class_counts)) / total_instances
weighted_recall = sum(r * n for r, n in zip(recalls, class_counts)) / total_instances

print(f"Weighted Precision: {weighted_precision:.4f}")
print(f"Weighted Recall: {weighted_recall:.4f}")



Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 104.0±68.5 MB/s, size: 234.5 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1wyO35GiaMELX5uUD8ZenhccOlW8RA1Dl/CS610_AML/yolo_traintestsplit/labels/test.cache... 41 images, 1 backgrounds, 0 corrupt: 100%|██████████| 42/42 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.04s/it]


                   all         42        198      0.883      0.882      0.873      0.805
                selfie         41         42      0.871      0.833      0.873      0.844
             bystander         41        156      0.895      0.931      0.872      0.767
Speed: 8.3ms preprocess, 21.3ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to runs/detect/val60
Mean Precision [across class](mp): 0.8829196047084471
Mean Recall [across class] (mr): 0.8823823861987028
mAP@0.5: 0.8726983413094558
mAP@0.5:0.95: 0.8054421107326887

Mean Precision/ Recall per Class
selfie: Precision=0.871, Recall=0.833, AP@0.5=0.873, AP@0.5:0.95=0.844
bystander: Precision=0.895, Recall=0.931, AP@0.5=0.872, AP@0.5:0.95=0.767

WEIGHTED PRECISION/RECALL
Weighted Precision: 0.8901
Weighted Recall: 0.9108
